In [ ]:
from pynq import Overlay
from pynq.lib import AxiGPIO
import time
import numpy as np
%matplotlib inline
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# 載入 Overlay
overlay = Overlay('rgb2yuv_verilog.bit')
ip = overlay.myip_0

In [ ]:
#讀入圖片
img = Image.open("mountain256.bmp").resize((256, 256)).convert("RGB")
r, g, b = img.split()
r = np.array(r).flatten()
g = np.array(g).flatten()
b = np.array(b).flatten()
plt.imshow(img, cmap='gray')
plt.title("RGB")
plt.show()

In [ ]:
# 建立輸出 buffer
y = np.zeros(256*256)
u = np.zeros(256*256)
v = np.zeros(256*256)

In [ ]:
ip.write(0x0, int(0)) # start
ip.write(0x0, int(1)) # start

v_start = time.time()
# 一個像素一個像素送入
for i in range(256*256):
    ip.write(0x4, int(r[i])) # R
    ip.write(0x8, int(g[i])) # G
    ip.write(0xC, int(b[i])) # B
    #time.sleep(0.0001)
    y[i] = ip.read(0x10) # Y
    u[i] = ip.read(0x14) # U
    v[i] = ip.read(0x18) # V
v_end = time.time()

In [ ]:
# reshape
y_reshape = y.reshape(256, 256)
u_reshape = u.reshape(256, 256)
v_reshape = v.reshape(256, 256)

In [ ]:
print("Verilog ip spen %f sec" %(v_end-v_start))
#顯示圖片
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.imshow(y_reshape, cmap='gray')
plt.title("Verilog Y")

plt.subplot(1, 3, 2)
plt.imshow(u_reshape, cmap='gray')
plt.title("Verilog U")

plt.subplot(1, 3, 3)
plt.imshow(v_reshape, cmap='gray')
plt.title("Verilog V")
plt.show()